In [20]:
## The program below was written by modifying the following code:
#    https://towardsdatascience.com/keras-101-a-simple-and-interpretable-neural-network-model-for-house-pricing-regression-31b1a77f05ae

In [21]:
# Set up

# Import dependencies
from sklearn.datasets import load_boston
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

# Dataset is the boston housing dataset from keras.
boston_data = load_boston()
boston_df = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston_df["MEDV"] = boston_data.target

boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [22]:
# Data is split for testing and training, then normalized
X = boston_df.loc[:, boston_df.columns != 'MEDV']
y = boston_df.loc[:, boston_df.columns == 'MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


# Normalization is done by replacing all X values with their normalized values
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [23]:
# Model is crafted
# Sequential is used to add depth, input shape uses the number of x_train columns, mean-square-error and linear activation are
#     used in the last layer as this is a linear regression.
model = Sequential()
model.add(Dense(128, input_shape=(13, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1792      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Model is trained
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
11/11 [==============================] - 0s 32ms/step - loss: 566.1932 - mae: 21.9466 - val_loss: 570.4389 - val_mae: 22.0165
Epoch 2/100
11/11 [==============================] - 0s 7ms/step - loss: 494.2345 - mae: 20.2555 - val_loss: 489.7285 - val_mae: 20.2391
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 401.1151 - mae: 17.8875 - val_loss: 378.8044 - val_mae: 17.5216
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 284.0717 - mae: 14.6169 - val_loss: 244.5691 - val_mae: 13.6216
Epoch 5/100
11/11 [==============================] - 0s 6ms/step - loss: 161.1678 - mae: 10.3806 - val_loss: 124.9990 - val_mae: 9.0392
Epoch 6/100
11/11 [==============================] - 0s 6ms/step - loss: 82.9413 - mae: 7.0103 - val_loss: 60.0932 - val_mae: 6.0739
Epoch 7/100
11/11 [==============================] - 0s 6ms/step - loss: 55.8729 - mae: 5.8215 - val_loss: 36.3385 - val_mae: 4.8054
Epoch 8/100
11/11 [==============================

In [25]:
# Visual representation to make a simpler way of verifying model is not overfitting
fig = go.Figure()

fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))

fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [26]:
# Visual representation to make a simpler way of verifying model using mean absolute error
fig = go.Figure()

fig.add_trace(go.Scattergl(y=history.history['mae'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_mae'],
                    name='Valid'))

fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Mean Absolute Error')

fig.show() 

In [27]:
# Model is tested
mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

5/5 [==============================] - 0s 3ms/step - loss: 18.5758 - mae: 2.6573
Mean squared error on test data:  18.575786590576172
Mean absolute error on test data:  2.6573002338409424
